In [2]:
# imports
import os
import obspy

def km2deg(km):
    deg = km / 111.3
    return deg

def inventory2traceid(inv, chancode=''):
    trace_ids = list()

    for networkObject in inv:
        if chancode:
            networkObject = networkObject.select(channel=chancode)
        stationObjects = networkObject.stations

        for stationObject in stationObjects:
            channelObjects = stationObject.channels
            for channelObject in channelObjects:
                this_trace_id = networkObject.code + '.' + stationObject.code + '.*.' + channelObject.code
                trace_ids.append(this_trace_id)
    
    return trace_ids

# constants
PRE_TRIGGER_SECS = 100
POST_TRIGGER_SECS = 300
LATITUDE = 28.557
LONGITUDE = -78.944
searchRadiusKm = 400
searchRadiusDeg = km2deg(searchRadiusKm)
ORIGIN_TIME = obspy.UTCDateTime(2024,2,8,3,48,7)
PRE_FILT = [0.01, 0.02, 15, 18] # defines a bandpass filter

all_streams = obspy.Stream()
startt = ORIGIN_TIME - PRE_TRIGGER_SECS
endt = ORIGIN_TIME + POST_TRIGGER_SECS
invs = []

from obspy.clients.fdsn import Client
for url in ["data.raspberryshake.org","service.iris.edu"]: 
    print(f'Trying to download data from fdsnws client to {url}')
    try:
        fdsnClient = Client(f'https://{url}') #, debug=True)
    except Exception as e:
        print(e)
        continue
    
    invfile = f'FL.{url}.xml'
    inv = None
    
    if os.path.isfile(invfile):
        print(f'reading inventory from {invfile}')
        inv = obspy.read_inventory(invfile)
    else:
        try:
            print(f'Trying to download an inventory for stations centred on ({LONGITUDE}, {LATITUDE}) radius {searchRadiusDeg}')
            inv = fdsnClient.get_stations( \
                starttime=startt, endtime=endt, \
                latitude="%.4f" % LATITUDE, longitude="%.4f" % LONGITUDE, maxradius="%.1f" % searchRadiusDeg, \
                level="response")
        except Exception as e:
            print(e)
            continue
        if inv:
            inv = inv.remove(network='SY') # get rid of synthetic data that annoying exists on IRIS
            print(f'Saving inventory to {invfile}')
            inv.write(invfile, format = 'stationxml')
    
    if inv:
        invs.append(inv)
        
        # get list of unique trace ids
        trace_ids = inventory2traceid(inv) 
        for trace_id in trace_ids:
            print(f'Processing {trace_id}')
            net, sta, loc, chan = trace_id.split('.')
            
            if not chan[0] in 'LMBJSHE': # only want data 20-250 Hz samplong rate
                print('- ignoring as sampling rate is too low')
                continue
            
            if not chan[1] in 'HD': # only want seismic or infrasound/barometric data
                print('- ignoring as not a seismic, infrasound, or barometric channel')
                continue
                
            print(f'Trying to get waveform(s) matching {net}.{sta}.{loc}.{chan} from {startt} to {endt}')
            try:
                stR = fdsnClient.get_waveforms(net, sta, loc, chan, startt, endt)
            except Exception as e:
                print(e)
                continue
            for tr in stR:
                tr.detrend()
                try:
                    print(f'Correcting {tr.id}')
                    tr.remove_response(inventory=inv, pre_filt=PRE_FILT, output="DISP",
                       water_level=60, plot=False) 
                except Exception as e:
                    print(e)
                    continue
                all_streams.append(tr)


if len(all_streams)>0:
    all_streams.merge()

    # Remove 1 Hz (L) channels if higher sampling rate available
    all_ids = [tr.id for tr in all_streams]
    for tr in all_streams:
        if tr.stats.channel[0]=='L':
            net,sta,loc,chan = tr.id.split('.')
            for band_code in 'MJBSHE':
                id_to_check = '.'.join([net, sta, loc, band_code + chan[1:]])
                if id_to_check in all_ids:
                    all_streams.remove(tr)

    # Save & Plot
    outfile = f'floridaquake.{startt}.mseed'
    print(f'Saving to {outfile}')
    all_streams.write(outfile, format='MSEED')
    all_streams.plot(equal_scale=False)
    
for inv in invs:
    print('Plotting an inventory map...this might take a while...')
    inv.plot(projection='local')

Trying to download data from fdsnws client to data.raspberryshake.org
No FDSN services could be discovered at 'https://data.raspberryshake.org'. This could be due to a temporary service outage or an invalid FDSN service address.
Trying to download data from fdsnws client to service.iris.edu
reading inventory from FL.service.iris.edu.xml
Processing IU.DWPF.*.BH1
Trying to get waveform(s) matching IU.DWPF.*.BH1 from 2024-02-08T03:46:27.000000Z to 2024-02-08T03:53:07.000000Z
Correcting IU.DWPF.00.BH1
Correcting IU.DWPF.10.BH1
Processing IU.DWPF.*.BH2
Trying to get waveform(s) matching IU.DWPF.*.BH2 from 2024-02-08T03:46:27.000000Z to 2024-02-08T03:53:07.000000Z
Correcting IU.DWPF.00.BH2
Correcting IU.DWPF.10.BH2
Processing IU.DWPF.*.BHZ
Trying to get waveform(s) matching IU.DWPF.*.BHZ from 2024-02-08T03:46:27.000000Z to 2024-02-08T03:53:07.000000Z
Correcting IU.DWPF.00.BHZ
Correcting IU.DWPF.10.BHZ
Processing IU.DWPF.*.HH1
Trying to get waveform(s) matching IU.DWPF.*.HH1 from 2024-02-08T0

ValueError: list.remove(x): x not in list